In [1]:
from skimage import io
from matplotlib import pyplot as plt
from PIL import Image
Image.MAX_IMAGE_PIXELS= None
%matplotlib qt   

import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from rasterio.transform import Affine
from rasterio.enums import Resampling
import numpy as np
import math
from scipy.ndimage import gaussian_filter
from rasterio.windows import from_bounds
from rasterio.windows import Window
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import entropy
import random

## Loading Data

In [8]:
import os 
os.getcwd()

'/Users/sking3/Desktop/Emily work'

In [9]:
capella = rasterio.open('Capella_CityG.tif')

In [10]:
sentinel = rasterio.open('Sentinel1_CityG.tif')

In [11]:
capella.bounds

BoundingBox(left=350069.1667, bottom=4102365.3308, right=358344.2167, top=4110486.0308)

In [12]:
sentinel.bounds

BoundingBox(left=350799.0494, bottom=4102135.0859, right=358359.0494, top=4111635.0859)

In [18]:
fig1 = plt.figure()
ax = fig1.add_subplot(111)
ax.imshow(sentinel.read(1))
plt.show()

## Plan A (Sentinel 1 & Capella)

**Step 1:** Downgrade Capella to pixel size $5m\times5m$

In [22]:
capella5_downgraded_band, capella5_pixel_w, capella5_pixel_h = bilinear_interpolation(capella, 5/0.35, 5/0.35)

In [24]:
# create_newtiff(capella, capella5_downgraded_band, capella5_pixel_w, capella5_pixel_h)

In [23]:
capella5_downgraded = rasterio.open('/Users/qiuqi/Downloads/capella5_downgraded.tif')

In [24]:
# using regrid function to do bilinear interpolation
band = capella.read(1)
ca_5 = regrid(band, 1655, 1624)
create_newtiff(capella, ca_5, capella5_pixel_w, capella5_pixel_h)
capella5_downgraded = rasterio.open('/Users/qiuqi/Downloads/New_output.tif')
capella5_downgraded.read(1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [25]:
sum(sum(capella5_downgraded.read(1)==capella5_downgraded.read(1))) # same output

2687720

**Step 2:** Upgraded Sentinel1 to pixel size $5m\times5m$

In [26]:
sentinel5_upgraded_band, sentinel5_pixel_w, sentinel5_pixel_h = bilinear_interpolation(sentinel, 0.5, 0.5)

In [281]:
# create_newtiff(sentinel, sentinel5_upgraded_band, sentinel5_pixel_w, sentinel5_pixel_h)

In [27]:
sentinel5_upgraded = rasterio.open('/Users/qiuqi/Downloads/sentinel5_upgraded.tif')

In [28]:
sentinel5_upgraded.bounds

BoundingBox(left=350799.0494, bottom=4102135.0859, right=358359.0494, top=4111635.0859)

In [57]:
capella5_downgraded.bounds

BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)

**Step 3:** Align downgraded Capella and upgraded Sentinel1

In [29]:
align(capella5_downgraded, sentinel5_upgraded)

In [62]:
sentinel5_upgraded.transform

Affine(5.0, 0.0, 350799.0494,
       0.0, -5.0, 4111635.0859)

In [61]:
rasterio.warp.aligned_target(sentinel5_upgraded.transform, sentinel5_upgraded.width, sentinel5_upgraded.height, capella5_downgraded.res)

(Affine(4.999999999999994, 0.0, 350794.9999999996,
        0.0, -4.999999999999828, 4111639.9999998584),
 1513,
 1902)

In [30]:
sentinel5_aligned = rasterio.open('/Users/qiuqi/Downloads/groundtruth.tif')

In [63]:
sentinel5_aligned.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 1512,
 'height': 1900,
 'count': 1,
 'crs': CRS.from_epsg(32637),
 'transform': Affine(4.999999999999994, 0.0, 350069.1667,
        0.0, -4.999999999999828, 4110486.0308)}

In [32]:
capella5_downgraded.bounds

BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)

**Step 4:** Extract overlapped pixels (or not?)

In [33]:
extract_overlap(sentinel5_aligned, capella5_downgraded)

BoundingBox(left=350069.1667, bottom=4100986.0308000003, right=357629.1667, top=4110486.0308)
BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)
(350069.1667, 4102366.0308000003, 357629.1667, 4110486.0308)
BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=357629.1667, top=4110486.0308)


**Step 5:** Prepare feature vector

In [34]:
sen_band = sentinel5_aligned.read(1)
cap_band = capella5_downgraded.read(1)

Training data(500)

In [35]:
train_index_vector, train_feature_vector = generate_feature_vector(sen_band, cap_band, 3, 500)
train_label_vector = generate_label_vector(capella5_downgraded.read(1), train_index_vector)

Testing data (100)

In [36]:
test_index_vector, test_feature_vector = generate_feature_vector(sen_band, cap_band, 3, 100)
test_label_vector = generate_label_vector(capella5_downgraded.read(1), test_index_vector)

**Step 6:** Train models

Random Forest

In [37]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_feature_vector, train_label_vector);
predictions = rf.predict(test_feature_vector)
# Calculate the absolute errors
errors = abs(predictions - test_label_vector)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_label_vector)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 23.35 degrees.
Accuracy: -inf %.


/var/folders/jt/fbh94k2946177w36lgmd11tw0000gn/T/ipykernel_29807/1734046684.py:10: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_label_vector)


## Plan B (Capella Only)

In [38]:
capella.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 23643,
 'height': 23202,
 'count': 4,
 'crs': CRS.from_epsg(32637),
 'transform': Affine(0.34999999999999953, 0.0, 350069.1667,
        0.0, -0.34999999999998793, 4110486.0308)}

In [39]:
capella10_downgraded_band, downgraded_pixel_w, downgraded_pixel_h = bilinear_interpolation(capella, 10/0.35, 10/0.35)

In [40]:
capella10_downgraded_band = regrid(capella.read(1), 828, 812)

In [41]:
create_newtiff(capella, capella10_downgraded_band, 10, 10)

In [48]:
capella10_downgraded = rasterio.open('/Users/qiuqi/Downloads/capella10_downgraded.tif')
# show_tiff(capella_downgraded)

In [46]:
capella10_downgraded = rasterio.open('/Users/qiuqi/Downloads/New_output.tif')

**Step 2:** Compare entropy with real sentinel1, add proper gaussian blur

Sentinel1 entropy

In [43]:
sen_en = compute_entropy(sentinel.read(1))
sen_en

0.5782087989992741

Blurred capella (fake sentinel1) entropy

In [44]:
find_proper_sigma(capella10_downgraded_band, sen_en)

2.5

In [56]:
compute_entropy(capella10_downgraded_band)

0.5114612240474095

In [54]:
band_blurred = gaussian_filter(capella10_downgraded_band, sigma=0.8)
compute_entropy(band_blurred)

0.3167977759745089

In [55]:
create_newtiff(capella, band_blurred, 10, 10)
capella10_blur = rasterio.open('/Users/qiuqi/Downloads/capella10_blur.tif')
# show_tiff(capella10_blur)

**Step 3:** prepare consistent pixel feature matrix

In [51]:
capella_blur_band = capella10_blur.read(1)

In [476]:
preprocessed_band = regrid(capella_blur_band, 23657, 23200)

KeyboardInterrupt: 

In [53]:
create_newtiff(capella, preprocessed_band, 0.35, 0.35)

In [55]:
capella_preprocessed = rasterio.open('/Users/qiuqi/Downloads/New_output.tif')

(0.35, 0.35)

**Step 4:** Prepare feature vector

In [ ]:
capella_feature_vectors = get_feature_vectors(preprocessed_band, 3)

**Step 5:** Train models

## Functions

In [17]:
def regrid(data, out_x, out_y):
    m = max(data.shape[0], data.shape[1])
    y = np.linspace(0, 1.0/m, data.shape[0])
    x = np.linspace(0, 1.0/m, data.shape[1])
    interpolating_function = RegularGridInterpolator((y, x), data)

    yv, xv = np.meshgrid(np.linspace(0, 1.0/m, out_y), np.linspace(0, 1.0/m, out_x))

    return interpolating_function((xv, yv))

In [16]:
def create_newtiff(original_dt, new_band, xres, yres):
    
    out_meta = original_dt.meta.copy()

    scaling_factor_width = xres / original_dt.res[0]
    scaling_factor_height = yres / original_dt.res[1]
    
    new_width = int(round(out_meta['width'] / scaling_factor_width))
    new_height = int(round(out_meta['height'] / scaling_factor_height))

    current_affine = original_dt.transform
    new_affine = rasterio.Affine(
        current_affine.a * scaling_factor_width,
        current_affine.b,
        current_affine.c,
        current_affine.d,
        current_affine.e * scaling_factor_height,
        current_affine.f
    )
    out_meta['transform'] = new_affine

    # Update image dimensions
    out_meta['width'] = new_width
    out_meta['height'] = new_height
    
    out_meta.update({'driver':'GTiff',
                 'count':1,
                 'dtype':'uint8',
                 'crs':original_dt.crs, 
                 'nodata':0.0})
    with rasterio.open('/Users/qiuqi/Downloads/New_output.tif', 'w', **out_meta) as dst:
        dst.write(new_band,1)

In [15]:
def resample_dataset(original_dt, xres, yres):
   
    scaling_factor_x = original_dt.transform.a / xres
    scaling_factor_y = original_dt.transform.e / yres

    new_width = max(int(round(original_dt.width * scaling_factor_x)), 1)
    new_height = max(int(round(original_dt.height * scaling_factor_y)), 1)

    out_meta = original_dt.meta.copy()
    out_meta.update({
        'width': new_width,
        'height': new_height,
        'transform': rasterio.Affine(xres, 0, original_dt.transform.c,
                                     0, yres, original_dt.transform.f)
    })


    with rasterio.open('/Users/qiuqi/Downloads/Trial_output.tif', 'w', **out_meta) as dst:
        data = original_dt.read(out_shape=(original_dt.count, new_height, new_width),
                                resampling=Resampling.bilinear)
        dst.write(data)

# with rasterio.open('path_to_original.tif', 'r') as original_dt:
#     resample_dataset(original_dt, 'path_to_resampled_band.tif', xres=10, yres=10)


In [14]:
def read_tiff(dataset):
    
    band = dataset.read(1)
    width = dataset.width
    height = dataset.height
    metadata = dataset.meta
    
    return band, width, height, metadata

In [13]:
def show_tiff(dataset):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.imshow(dataset.read(1), cmap='gray')

In [12]:
def compare_tiff(dataset1, dataset2):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(121)
    ax.imshow(dataset1.read(1), cmap='pink')
    ax = fig.add_subplot(122)
    ax.imshow(dataset2.read(1), cmap='pink')

In [11]:
def bilinear_interpolation(original_dt, h_scale_factor, w_scale_factor):
    
    band = original_dt.read(1)
    
    old_w=original_dt.width
    old_h=original_dt.height

    new_w = round(old_w/w_scale_factor)
    new_h = round(old_h/h_scale_factor)

    resized = np.zeros((new_h, new_w))

    for i in range(new_h):
        for j in range(new_w):
            # Original coordinates 
            x = i * h_scale_factor
            y = j * w_scale_factor
            # Coordinate values for 4 surrounding pixels.
            x_floor = math.floor(x)
            x_ceil = min(old_h - 1, math.ceil(x))
            y_floor = math.floor(y)
            y_ceil = min(old_w - 1, math.ceil(y))
    
            if (x_ceil == x_floor) and (y_ceil == y_floor):
                # Estimated pixel value
                q = band[int(x), int(y)]
            elif (x_ceil == x_floor):
                q1 = band[int(x), int(y_floor)]
                q2 = band[int(x), int(y_ceil)]
                q = q1 * (y_ceil - y) + q2 * (y - y_floor)
            elif (y_ceil == y_floor):
                q1 = band[int(x_floor), int(y)]
                q2 = band[int(x_ceil), int(y)]
                q = (q1 * (x_ceil - x)) + (q2 * (x - x_floor))
            else:
                # neighbouring pixel values
                v1 = band[x_floor, y_floor]
                v2 = band[x_ceil, y_floor]
                v3 = band[x_floor, y_ceil]
                v4 = band[x_ceil, y_ceil]
    
                # Estimated pixel value
                q1 = v1 * (x_ceil - x) + v2 * (x - x_floor)
                q2 = v3 * (x_ceil - x) + v4 * (x - x_floor)
                q = q1 * (y_ceil - y) + q2 * (y - y_floor)
            
            if (q<0):
                q=0
            elif (q>255):
                q=255
            
            resized[i,j] = q
            
    new_pixel_w = w_scale_factor*original_dt.res[0]
    new_pixel_h = h_scale_factor*original_dt.res[1]
    return resized.astype(np.uint8), new_pixel_w, new_pixel_h

In [10]:
def generate_feature_vector(feature_band, label_band, window_size, training_num):
    index_vector = []
    for n in range(training_num):
        index = [random.randrange(1,label_band.shape[0]), random.randrange(1,label_band.shape[1])]
        index_vector.append(index)
    feature_vectors = get_feature_vectors(feature_band, window_size, index_vector)
    return index_vector, feature_vectors.flatten().reshape((training_num, window_size*window_size))

In [9]:
def get_feature_vectors(pixel_array, window_size, pixel_index):
    
    if window_size % 2 == 0:
        raise ValueError("Window size should be an odd number.")

    half_window = window_size // 2
    height, width = pixel_array.shape
    feature_vectors = np.zeros((len(pixel_index), 1, window_size * window_size))

    for n in range(len(pixel_index)):
        row = pixel_index[n][0]  
        col = pixel_index[n][1]
        # Define the window around the current pixel
        row_start = max(0, row - half_window)
        row_end = min(height, row + half_window + 1)
        col_start = max(0, col - half_window)
        col_end = min(width, col + half_window + 1)

            # Extract pixel values within the window
        window_values = pixel_array[row_start:row_end, col_start:col_end].ravel()

            # Pad the window values to fill the feature vector
        padded_values = np.pad(window_values, (0, window_size * window_size - len(window_values)), mode='constant')

            # Store the feature vector for the current pixel
        feature_vectors[n, 0, :] = padded_values

    return feature_vectors

In [8]:
def generate_label_vector(band, index_vector):
    label_vector = []
    for n in range(len(index_vector)):
        row = index_vector[n][0]
        column = index_vector[n][1]
        
        label = band[row, column]
        label_vector.append(label)
    return label_vector

In [7]:
def compute_entropy(band):
    
    dt = band.flatten()
    nonzero_dt = dt [dt !=0]
    length = len(nonzero_dt)
    pk = np.full(length, 1/256)
    en = entropy(pk, nonzero_dt)
    return en

In [6]:
def find_proper_sigma(input_band, desire_entropy):
    sig = 0
    blurred_band = gaussian_filter(input_band, sigma=sig)
    en = compute_entropy(blurred_band)
#     print(abs(en - desire_entropy))
    while (abs(en - desire_entropy) > 0.01):
        sig += 0.5
        blurred_band = gaussian_filter(input_band, sigma=sig)
        en = compute_entropy(blurred_band)
#         print(abs(en - desire_entropy))
    return sig

In [5]:
def align(raster, labels):
    transform, width, height = calculate_default_transform(
    labels.crs, raster.crs, labels.width, labels.height, *labels.bounds)
    
    kwargs = labels.meta.copy()
    kwargs.update({
    'crs': raster.crs,
    'transform': raster.transform,
    'width':labels.width,
    'height':labels.height
    })

    with rasterio.open('/Users/qiuqi/Downloads/groundtruth.tif', 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(labels, 1),
            destination=rasterio.band(dst, 1),
            src_transform=labels.transform,
            src_crs=labels.crs,
            dst_transform=raster.transform,
            dst_crs=raster.crs,
            resampling=Resampling.nearest)

In [72]:
def align(raster, labels): #### trial
    transform, width, height = rasterio.warp.aligned_target(labels.transform, labels.width, labels.height, raster.res)
#     transform, width, height = calculate_default_transform(
#     labels.crs, raster.crs, labels.width, labels.height, *labels.bounds)
    print(transform, width, height)
    kwargs = labels.meta.copy()
    kwargs.update({
    'crs': raster.crs,
    'transform': transform,
    'width': width,
    'height':height
    })

    with rasterio.open('/Users/qiuqi/Downloads/groundtruth.tif', 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(labels, 1),
            destination=rasterio.band(dst, 1),
            src_transform=labels.transform,
            src_crs=labels.crs,
            dst_transform=raster.transform,
            dst_crs=raster.crs,
            resampling=Resampling.nearest)
        


In [335]:
sentinel.bounds

BoundingBox(left=350799.0494, bottom=4102135.0859, right=358359.0494, top=4111635.0859)

In [354]:
sentinel5_upgraded.bounds

BoundingBox(left=350799.0494, bottom=4102135.0859, right=358359.0494, top=4111635.0859)

In [350]:
sentinel5_aligned.bounds

BoundingBox(left=350069.1667, bottom=4100986.0308000003, right=357629.1667, top=4110486.0308)

In [73]:
align(capella5_downgraded, sentinel5_upgraded)
sentinel5_al = rasterio.open('/Users/qiuqi/Downloads/groundtruth.tif')
sentinel5_al.meta

| 5.00, 0.00, 350795.00|
| 0.00,-5.00, 4111640.00|
| 0.00, 0.00, 1.00| 1513 1902


{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 1513,
 'height': 1902,
 'count': 1,
 'crs': CRS.from_epsg(32637),
 'transform': Affine(4.999999999999994, 0.0, 350794.9999999996,
        0.0, -4.999999999999828, 4111639.9999998584)}

In [74]:
sentinel5_al.bounds

BoundingBox(left=350794.9999999996, bottom=4102129.999999859, right=358359.9999999996, top=4111639.9999998584)

In [75]:
sentinel5_aligned.bounds

BoundingBox(left=350069.1667, bottom=4100986.0308000003, right=357629.1667, top=4110486.0308)

In [69]:

sentinel5_aligned = rasterio.open('/Users/qiuqi/Downloads/sentinel5_aligned.tif')
sentinel5_aligned.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 1512,
 'height': 1900,
 'count': 1,
 'crs': CRS.from_epsg(32637),
 'transform': Affine(4.999999999999994, 0.0, 350069.1667,
        0.0, -4.999999999999828, 4110486.0308)}

In [347]:
capella.bounds

BoundingBox(left=350069.1667, bottom=4102365.3308, right=358344.2167, top=4110486.0308)

In [348]:
capella5_downgraded.bounds

BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)

In [342]:
extract_overlap(sentinel5_aligned, capella5_downgraded)

In [76]:
extract_overlap(sentinel5_al, capella5_downgraded)

BoundingBox(left=350794.9999999996, bottom=4102129.999999859, right=358359.9999999996, top=4111639.9999998584)
BoundingBox(left=350069.1667, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)
(350794.9999999996, 4102366.0308000003, 358344.1667, 4110486.0308)
BoundingBox(left=350794.9999999996, bottom=4102366.0308000003, right=358344.1667, top=4110486.0308)


In [79]:
output = rasterio.open('/Users/qiuqi/Downloads/output_overlap.tif')
output.bounds

BoundingBox(left=350794.9999999996, bottom=4102366.0308000008, right=358339.9999999996, top=4110486.0308000003)

In [4]:
def extract_overlap(src1, src2):
    bounds1 = src1.bounds
    bounds2 = src2.bounds
    print(bounds1)
    print(bounds2)
    # Calculate the overlapping extent
    overlap_extent = (
        max(bounds1.left, bounds2.left),
        max(bounds1.bottom, bounds2.bottom),
        min(bounds1.right, bounds2.right),
        min(bounds1.top, bounds2.top)
    )
    print(overlap_extent)
    oe = rasterio.coords.BoundingBox(*overlap_extent)
    print(oe)
    # Read the overlapping area from the larger GeoTIFF (file1.tif)
    window = src1.window(*oe)
    data1 = src1.read(1, window=window)

    # Create a new GeoTIFF file to save the extracted overlapping area
    output_filename = '/Users/qiuqi/Downloads/output_overlap.tif'
    profile = src1.profile
    profile.update(width=window.width, height=window.height, transform=src1.window_transform(window))
    with rasterio.open(output_filename, 'w', **profile) as dst:
        dst.write(data1, 1)


In [3]:
def extract_overlap(src1, src2):
    data1 = src1.read(1)

    # extract metadata from the first raster
    meta = src1.meta.copy()

    # read the window of the second raster with the same extent as the first raster
    window = src1.window(*src2.bounds)

    # read the data from the second raster with the same window as first raster
    data1 = src1.read(1, window=window, boundless=True, fill_value=0)
    data1 = np.where(data1 == src1.nodata, 0, data1)
    
    with rasterio.open('/Users/qiuqi/Downloads/trial_overlap.tif', 'w', **meta) as dst:
        dst.write(data1, 1)

In [2]:
def extract_overlap(src1, src2):
    data1 = src1.read(1)

    # extract metadata from the first raster
    meta = src1.meta.copy()

    # read the window of the second raster with the same extent as the first raster
    window = src2.window(*src1.bounds)

    # read the data from the second raster with the same window as first raster
    data2 = src2.read(1, window=window, boundless=True, fill_value=0)
    data2 = np.where(data2 == src2.nodata, 0, data2)

    data = data1 - data2
    # write the result to a new raster
    with rasterio.open('/Users/qiuqi/Downloads/trial_overlap.tif', 'w', **meta) as dst:
        dst.write(data,1)